In [1]:
import pandas as pd
import lightgbm as lgb
import numpy as np
import gc
import itertools

from importlib import reload
import date
reload(date)
from date import *
import model
reload(model)
from model import *
from metric import get_weights, NWRMSLE_log

In [2]:
df = load_data_in_date_range('./data/train_processed.csv', '2017-04-04', '2017-08-15', 110000000)

Reading batch from position 110000000, batch size 10000000...
Filtering 10000000...
Filtered 8659998, mapping...
Mapped, reducing...
Batch done.
Reading batch from position 120000000, batch size 10000000...
Filtering 5497041...
Filtered 5497041, mapping...
Mapped, reducing...
Batch done.
End of dataset is found.


In [3]:
print('Reading additional datasets')
items = pd.read_csv('./data/items_encoded.csv')
stores = pd.read_csv('./data/stores_encoded.csv')

Reading additional datasets


In [4]:
print('Converting data and joining additional data')
df = convert_unit_sales(df)
df = fill_empty_sales(df)
df, cols_categories = extend_dataset(df, items, stores)
gc.collect()

Converting data and joining additional data


205

In [5]:
cat_features = items_cols + stores_cols + ['store_nbr','item_nbr','onpromotion']
combinations = list(itertools.combinations(cat_features, 2)) + \
list(itertools.combinations(cat_features, 1)) + \
[['store_nbr','item_nbr','onpromotion']] #, ['store_nbr','item_nbr']]
print('Adding mean target encoding')
df, cols_mean = add_mean_encoding(df, combinations)
gc.collect()
df.head()

Adding mean target encoding


,date,store_nbr,item_nbr,id,unit_sales,onpromotion,weekday,weekend,salary,family,...,mean_unit_sales_by_(perishable),mean_unit_sales_by_(family),mean_unit_sales_by_(city),mean_unit_sales_by_(type),mean_unit_sales_by_(cluster),mean_unit_sales_by_(state),mean_unit_sales_by_(store_nbr),mean_unit_sales_by_(item_nbr),mean_unit_sales_by_(onpromotion),mean_unit_sales_by_(store_nbr+item_nbr+onpromotion)
0,1576,46,1695836,113723318.0,3.983762,True,2,False,False,30,...,0.954195,1.029249,1.098829,1.236079,1.307416,1.090374,1.352238,1.758596,1.969504,3.759474
1,1577,46,1695836,113824033.0,2.771612,False,3,False,False,30,...,0.954195,1.029249,1.098829,1.236079,1.307416,1.090374,1.352238,1.758596,0.749576,2.219200
2,1578,46,1695836,113927633.0,3.089621,False,4,False,False,30,...,0.954195,1.029249,1.098829,1.236079,1.307416,1.090374,1.352238,1.758596,0.749576,2.219200
3,1579,46,1695836,114042179.0,3.789149,False,5,True,False,30,...,0.954195,1.029249,1.098829,1.236079,1.307416,1.090374,1.352238,1.758596,0.749576,2.219200
4,1580,46,1695836,114155535.0,4.023117,False,6,True,True,30,...,0.954195,1.029249,1.098829,1.236079,1.307416,1.090374,1.352238,1.758596,0.749576,2.219200


In [6]:
print('Adding lagged features')
df_prev = df[['item_nbr', 'store_nbr', 'date', 'unit_sales']]
df, cols_lagged = fill_lagged(df, df_prev, 12, 16, True)
del df_prev
gc.collect()
!telegram-send "Lagged and mean extraction is ready, starting validation."

Adding lagged features
Processing lagged 12...


/home/ubuntu/.local/lib/python3.5/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/home/ubuntu/grocery/model.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_prev[colname] = df_prev['unit_sales']


Lagged 12 is done.
Processing lagged 13...
Lagged 13 is done.
Processing lagged 14...
Lagged 14 is done.
Processing lagged 15...
Lagged 15 is done.
Processing lagged 16...
Lagged 16 is done.


MemoryError: 

In [ ]:
df.to_hdf('./data/train_extended.hdf')

In [ ]:
folds = [('2017-05-01', '2017-06-15', '2017-06-16', '2017-06-30'),
         ('2017-05-16', '2017-06-30', '2017-07-01', '2017-07-15'),
         ('2017-06-01', '2017-07-15', '2017-07-16', '2017-07-31'),
         ('2017-06-16', '2017-07-31', '2017-08-01', '2017-08-15')]

param = {
    'num_leaves':30,
    'objective':'regression_l2',
    'metric':'l2_root',
    'num_threads':4
        }

In [ ]:
results = []
bsts = []
x_cols = cols_categories + ['onpromotion'] + cols_lagged + cols_mean
print('X cols: {}'.format(x_cols))

for train_start, train_end, test_start, test_end in folds:
    print("Extracting fold...")
    train = extract_by_date(df, train_start, train_end)
    test= extract_by_date(df, test_start, test_end)
    
    print("Preparing train...")
    train_X = train[x_cols]
    train_y = train['unit_sales']
    train_weights = get_weights(train['item_nbr'])
    train_dataset = lgb.Dataset(train_X, label=train_y, weight=train_weights)
    #del train
    
    print("Preparing test...")
    test_X = test[x_cols].fillna(0.0)
    test_y = test['unit_sales']
    test_weights = get_weights(test['item_nbr'])
    test_dataset = lgb.Dataset(test_X, label=test_y, weight=test_weights, reference=train_dataset)
    #del test
    
    print("Training!")
    bst = lgb.train(param, 
                    train_dataset, 
                    200,
                    valid_sets=[test_dataset], 
                    early_stopping_rounds=10, 
                    verbose_eval=True, 
                    feature_name=x_cols, 
                    categorical_feature=ext_cols)
    
#     test_y_pred = bst.predict(test_X)
#     error = NWRMSLE_log(test_y_pred, test_y, test_weights)
#     print('Validation error: {}'.format(error))
    
    results.append((bst.best_iteration, error))
    bsts.append(bst)

In [ ]:
!telegram-send "Mean lagged xgb validation finished. Results: $results"

In [ ]:
bsts[0]

In [ ]:
param['task'] = 'prediction'
start = '2017-07-01'
end = '2017-08-15'
num_round = 100
train = extract_by_date(df, start, end)
train, ext_cols = extend_dataset(train, items, stores)
x_cols = ext_cols + ['onpromotion'] + lagged_cols
train_X = train[x_cols]
train_y = train['unit_sales']
train_weights = get_weights(train['item_nbr'])
train_dataset = lgb.Dataset(train_X, label=train_y, weight=train_weights)
bst = lgb.train(param, 
                train_dataset,
                num_round,
                feature_name=x_cols, 
                categorical_feature=ext_cols)

C:\Users\Anatoliy\Anaconda2\envs\py35\lib\site-packages\lightgbm\basic.py:1007: UserWarning: categorical_feature in Dataset is overrided. New categorical_feature is ['city', 'class', 'cluster', 'family', 'perishable', 'salary', 'state', 'type', 'weekday', 'weekend']
  warnings.warn('categorical_feature in Dataset is overrided. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


In [ ]:
test = pd.read_csv('./data/test_processed.csv', dtype=types)

df_prev = extract_by_date(df, '2017-07-25', '2017-08-15')
test = fill_mean_encoding(test, df_prev)
test = fill_lagged(test, df_prev, 12, 18)

test, ext_cols = extend_dataset(test, items, stores)
x_cols = ext_cols + ['onpromotion'] + lagged_cols
test_X = test[x_cols]
test['unit_sales'] = bst.predict(test_X)

In [ ]:
test.sort_values(by='id', inplace=True)
test.ix[test.unit_sales < 0, 'unit_sales'] = 0
test[['id', 'unit_sales']].to_csv('./submissions/lgb_mean_encoded_lagged_0.54.csv.gz', float_format="%.4f", index=False, compression='gzip')
!telegram-send "Submission done."